# Process Plink Results

Read plink results and keep only the test results for the HLA alleles.

Julia made a notebook that sort of does this too, but I'm going to leave this one
because it makes a single file that is nice to have.

In [1]:
import glob
import os
import sys

import pandas as pd

import cdpybio as cpb

In [2]:
outdir = os.path.realpath(os.path.join('../output/process_plink_results'))
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [3]:
traits = pd.read_table('../data/traits.tsv', header=0, index_col=0)
# Rename cancer codes to match codes that were used in HLA analysis.
traits.index = [x.replace('cancer', '') for x in traits.index]

In [6]:
fns = glob.glob('/oak/stanford/groups/mrivas/users/jolivier/repos/hla-assoc/data/PLINK_results/*hybrid')
codes = [os.path.split(x)[1].split('.')[0] for x in fns]

out_fn = os.path.join(outdir, 'plink_results_all.tsv.gz')
if not os.path.exists(out_fn):
    dfs = []
    for fn in fns:
        t = cpb.plink.read_logistic2(fn)
        t['code'] = os.path.split(fn)[1].split('.')[0]
        t = t[['code', 'FIRTH?', 'TEST', 'OBS_CT', 'OR', 'SE', 'T_STAT', 'P']]
        dfs.append(t)
    results = pd.concat(dfs)
    results.to_csv(out_fn, sep='\t', compression='gzip')
else:
    results = pd.read_table(out_fn)

In [7]:
# Filter results by allele and disease frequency
hla = pd.read_table('/oak/stanford/groups/mrivas/ukbb/24983/hla/ukb_hla_v2.txt')
covar = pd.read_table('/oak/stanford/groups/mrivas/ukbb/24983/phe_qc/ukb24983_GWAS_covar.phe', 
                      index_col=0)
hla.index = covar.index
remove = pd.read_table('/oak/stanford/groups/mrivas/ukbb/24983/phe_qc/ukb24983_remove.phe',
                       index_col=0, header=None, squeeze=True)
hla = hla.drop(remove)

In [11]:
# Note: this relies on this additive_assoc_adj_p_all.csv file because I'm not
# exactly sure where the original list of phenotypes came from. It doesn't
# really matter, I just want to make sure I know which phenotypes were included.
additive_res = pd.read_csv('../manuscript/additive_assoc_adj_p_all.csv', index_col=0)

shared = list(set(codes) & set(traits.index))
missing = list(set(additive_res.index) - set(traits[traits['numcases'] >= 500].index))
print(traits.loc[missing])

        regtype category  numcases                         phenotype
HC69   logistic       HC     482.0                polycythaemia_vera
HC432  logistic       HC     487.0             mitral_valve_prolapse
HC421  logistic       HC     478.0           other_abdominal_problem
HC352  logistic       HC     474.0  systemic_lupus_erythematosis/sle
HC12   logistic       HC     492.0  testicular_problems_(not_cancer)


It seems that the number of cases for some of the diseases is less than 500
according to the counts I have from the gcorr app. Julia made a mistake 
when calculating the disease frequencies in the `check_firth` notebook. These diseases
are ones that were included but shouldn't have been if we took 500 as a cutoff.

In [20]:
print(traits.loc[list(set(traits[traits.numcases >= 470].index) & set(codes) - 
                      set(additive_res.index))])

        regtype category  numcases           phenotype
HC278  logistic       HC     477.0   cerebral_aneurysm
HC375  logistic       HC     477.0  alcohol_dependency
HC256  logistic       HC     476.0        fracture_toe


These are three phenotypes we could've included if we used a cutoff of 470. I think
it's fine to leave these out since the cutoff was arbitrary anyway.

In [34]:
phenos = list(additive_res.index)

In [25]:
fn = os.path.join("../private_output/print_rds/ukb_hla_v2_rounded_remove.txt")
dosages = pd.read_csv(fn, index_col=0)
allele_freqs = dosages.sum() / float(dosages.shape[0])
alleles = list(allele_freqs[allele_freqs >= 0.001].index)

In [36]:
with open(os.path.join(outdir, 'phenos.txt'), 'w') as f:
    f.write('\n'.join(phenos) + '\n')

In [37]:
with open(os.path.join(outdir, 'alleles.txt'), 'w') as f:
    f.write('\n'.join(alleles) + '\n')

In [39]:
results.head()

,ID,code,FIRTH?,TEST,OBS_CT,OR,SE,T_STAT,P
0,A_101,HC250,N,ADD,336752,0.930059,0.111899,-0.647970,0.517004
1,A_101,HC250,N,age,336752,1.032830,0.008134,3.970880,0.000072
2,A_101,HC250,N,sex,336752,1.354470,0.122521,2.476410,0.013271
3,A_101,HC250,N,PC1,336752,0.957894,0.039932,-1.077290,0.281349
4,A_101,HC250,N,PC2,336752,1.028930,0.041295,0.690724,0.489739


In [41]:
len(set(results['ID']) & set(alleles))

175

In [42]:
len(set(results['code']) & set(phenos))

270

In [46]:
adj_pvals = pd.read_csv('../output/compare_cutoff_pvals/adj_p_vals.csv', index_col=0)
adj_pvals = adj_pvals.stack().sort_index()

In [71]:
results_f = results[results['code'].isin(phenos)]
results_f = results_f[results_f['ID'].isin(alleles)]
results_f = results_f[results_f['TEST'] == 'ADD']
results_f = results_f.set_index(['code', 'ID']).sort_index()
results_f['adj_pval'] = adj_pvals
out_fn = os.path.join(outdir, 'plink_results_filtered.tsv.gz')
results_f.to_csv(out_fn, sep='\t', compression='gzip')

out_fn = os.path.join(outdir, 'plink_results_filtered_sig.tsv.gz')
results_sig = results_f[results_f.adj_pval < 0.05]
results_sig.to_csv(out_fn, sep='\t', compression='gzip')